# Battle of Neighborhoods capstone project


## Data sources



Data to be taken from Forsquare for each area:

- caffes/restaurants and thei types
- bars/clubs (nightlife)

From other sources (e.g. OSM)

- public transportation
- schools
- parks
- churches 

using venues categories from this list https://developer.foursquare.com/docs/build-with-foursquare/categories/ we are going to calculate number of each venue to population ratio which will give us availability of such type of venues.

Population data as well as crime data is going to be taken from open sources like NYC open data (https://data.cityofnewyork.us/) and Toronto open data (https://data.torontopolice.on.ca/ - police department, https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-gallery/ - open data of Toronto city)

Market rent data for both cities will be taken: 
- for Toronto - data from Canada Mortgage and Housing Corporation https://eppdscrmssa01.blob.core.windows.net/cmhcprodcontainer/sf/project/archive/data_tables/data_tables/rental_market_report_data_tables_greater_toronto_area/rmr-toronto-2020-en.xlsx
- for NYC - data from StreetEasy.com https://streeteasy-market-data-download.s3.amazonaws.com/rentals/All/medianAskingRent_All.zip

In [2]:
# The code was removed by Watson Studio for sharing.

# Data review

## Forsquare data

Forsqare will be used for exploration nightlife and caffe/restaurants data and such search as below will be iterated for each neighborhood in both cities

For example:

In [3]:
lat = 43.639397
lng = -79.385605

radius=1500
categoryid = '4d4b7105d754a06376d81259' #category = Nightlife Spot
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryid={}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            categoryid,
            radius
            )
print(url)
results = requests.get(url).json()["response"]['venues']


https://api.foursquare.com/v2/venues/search?&client_id=00YBS1EPBQG4QMI4ZDGEH5SCHQHVQQMMGLIB5WOCVHXPX1L5&client_secret=YMUPUMTOI0RMEJ4K1TBGEPY2QMGIYMBL5LFG2NQJWL0H4P0P&v=20180605&ll=43.639397,-79.385605&categoryid=4d4b7105d754a06376d81259&radius=1500


In [4]:
venues_list=[]
venues_list.append([(
            v['location']['lat'],
            v['location']['lng'],
            v['location']['formattedAddress'][0],np.nan,
            v['name']
) for v in results])
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.drop_duplicates(subset=[0,1], keep='first', inplace = True)
nearby_venues.drop_duplicates(subset=[2], keep='first', inplace = True, ignore_index = True)
#nearby_venues[3] = np.nan
nearby_venues

,0,1,2,3,4
0,43.638765,-79.385871,249 Queen's Quay West,NaN,Radisson Admiral Hotel Toronto-Harbourfront
1,43.638910,-79.384851,Queens Quay W. (Btwn Spadina & Lower Simcoe),NaN,Harbourfront Wavedecks
2,43.639162,-79.386128,266 Queens Quay W,NaN,Swiss Chalet
3,43.639210,-79.385769,260 Queens Quay W (at Rees St),NaN,260 Queens Quay W
4,43.638837,-79.385186,Queens Quay W. (in the Radisson) (Rees Street),NaN,Starbucks
5,43.639270,-79.385390,254 Queens Quay W,NaN,Harbour Nails
6,43.645167,-79.380641,65 Front St W (btwn Bay & York St),NaN,Union Station
7,43.639699,-79.382691,Toronto ON M5J 2Y6,NaN,Shatter Abbas
8,43.639397,-79.384452,Toronto ON,NaN,Rees Street Parkette
9,43.638985,-79.385687,Canada,NaN,CitySightSeeing Toronto


In [5]:
# create map using latitude and longitude values
tor_latitude = lat
tor_longitude = lng
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=15)

# add markers to map
for lat, lng, in zip(nearby_venues[0], nearby_venues[1]):
    label = '({}), {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


## Open Street Map  data

Forsqare service originally was designed for venue recommendation which makes forsquare weak if there is a need to find and count POIs like bus stops, schools and parks.

That's why for such kind of search I decided to switch to OSM data. 

For example for schools

### Result from Forsquare vs OSM

In [6]:
lat = 43.639397
lng = -79.385605

radius=1500
categoryid = '4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106' #category = Elementary and middle schools
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryid={}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            categoryid,
            radius
            )
print(url)
results = requests.get(url).json()["response"]['venues']
venues_list=[]
venues_list.append([(
            v['location']['lat'],
            v['location']['lng'],
            v['location']['formattedAddress'][0],np.nan,
            v['name']
) for v in results])
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.drop_duplicates(subset=[0,1], keep='first', inplace = True)
nearby_venues.drop_duplicates(subset=[2], keep='first', inplace = True, ignore_index = True)
tor_latitude = lat
tor_longitude = lng
map_toronto = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=15)

# add markers to map
for lat, lng, in zip(nearby_venues[0], nearby_venues[1]):
    label = '({}), {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    


https://api.foursquare.com/v2/venues/search?&client_id=00YBS1EPBQG4QMI4ZDGEH5SCHQHVQQMMGLIB5WOCVHXPX1L5&client_secret=YMUPUMTOI0RMEJ4K1TBGEPY2QMGIYMBL5LFG2NQJWL0H4P0P&v=20180605&ll=43.639397,-79.385605&categoryid=4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106&radius=1500


In [7]:
overpass_url = "https://lz4.overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node(around:1500, 43.639397, -79.385605)[amenity=school];
out center;
""".format(tor_latitude, tor_longitude)
resp = requests.get(overpass_url, 
                        params={'data': overpass_query})
#print(resp.text)
osm_data = resp.json()['elements']
osm_list=[]
osm_list.append([(
            v['lat'],
            v['lon'],
            '{} {}'.format(v['tags']['addr:housenumber'], v['tags']['addr:street']),
            np.nan
    ,
            v['tags']['name']
) for v in osm_data])
osm_venues = pd.DataFrame([item for osm_list in osm_list for item in osm_list])

for lat, lng, in zip(osm_venues[0], osm_venues[1]):
    label = '({}, {})'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Results from Forsquare are in blue and from OSM are in red. As you can see OSM is more precise

## Open data available online

The rest of the data is simply available online and will be used after transformation as is. For example NYC Census population data is available here

https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD

In [8]:
ny_pop_url = 'https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD'

nyc_pop_data = pd.read_csv(ny_pop_url)
nyc_pop_data = nyc_pop_data.loc[nyc_pop_data['Year'] == 2010]
nyc_pop_data.drop(columns=['Year', 'FIPS County Code','NTA Code'], inplace = True)
nyc_pop_data.head()

,Borough,NTA Name,Population
195,Bronx,Claremont-Bathgate,31078
196,Bronx,Eastchester-Edenwald-Baychester,34517
197,Bronx,Bedford Park-Fordham North,54415
198,Bronx,Belmont,27378
199,Bronx,Bronxdale,35538


## Conclusion on data

Data, that needed for analysis and clusterisation is fully available from different sources, but needs some additional processing to

- clean data
- merge data
- make data meaningful (as in case with Forsquare vs OSM)

